In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import os
from os import listdir
from scipy.misc import imread, imresize
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from datetime import datetime as dt

%matplotlib inline

c:\users\ryan\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## ================== preprocessing ======================== ##

# Settings:
img_size = 64
grayscale_images = True
num_class = 10
test_size = 0.2


def get_img(data_path):
    # Getting image array from path:
    img = imread(data_path, flatten=grayscale_images)
    img = imresize(img, (img_size, img_size, 1 if grayscale_images else 3))
    return img

def get_dataset(dataset_path='Dataset'):
    # Getting all data from data path:
    try:
        X = np.load('npy_dataset/X.npy')
        Y = np.load('npy_dataset/Y.npy')
        
        img = np.load('npy_dataset/X.npy')
        lbl = np.load('npy_dataset/Y.npy')

        data = []
        label = []
        for i in zip(img,lbl):
            data.append(i[0])
            data.append(np.flip(i[0],1))
            label.append(i[1])
            label.append(i[1])
            
    except:
        labels = listdir(dataset_path) # Geting labels
        X = []
        Y = []
        for i, label in enumerate(labels):
            datas_path = dataset_path+'/'+label
            for data in listdir(datas_path):
                img = get_img(datas_path+'/'+data)
                X.append(img)
                Y.append(i)
        # Create dateset:
        X = 1-np.array(X).astype('float32')/255.
        Y = np.array(Y).astype('float32')
        Y = to_categorical(Y, num_class)
        if not os.path.exists('npy_dataset/'):
            os.makedirs('npy_dataset/')
        np.save('npy_dataset/X.npy', X)
        np.save('npy_dataset/Y.npy', Y)
        
        img = np.load('npy_dataset/X.npy')
        lbl = np.load('npy_dataset/Y.npy')

        data = []
        label = []
        for i in zip(img,lbl):
            data.append(i[0])
            data.append(np.flip(i[0],1))
            label.append(i[1])
            label.append(i[1])
        # plt.imshow(data[500])
        # print(label[500])
        np.save('npy_dataset/X.npy', data)
        np.save('npy_dataset/Y.npy', label)
        
        
    return X, Y

X, Y = get_dataset()
print(len(Y))

4124


In [3]:
## COVNET

x = tf.placeholder(tf.float32, shape=[None, 64,64], name='X')
x_image = tf.reshape(x, [-1, 64, 64, 1])
# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [4]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name,stride=[1, 1, 1, 1]):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=stride, padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights
    
def new_pool_layer(input,name,stride=[1,2,2,1],ksize=[1,2,2,1]):
  with tf.variable_scope(name) as scope:
    layer = tf.nn.max_pool(input,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
    return layer

def new_relu_layer(input,name):
  with tf.variable_scope(name) as scope:
        layer = tf.nn.relu(input)
        return layer
    

def new_fc_layer(input, num_inputs, num_outputs, name):
  with tf.variable_scope(name) as scope:
      # Create new weights and biases.
      weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
      biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
      # Multiply the input and weights, and then add the bias-values.
      layer = tf.matmul(input, weights) + biases
      return layer

In [5]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=3, num_filters=6, name ="conv1")
layer_pool1 = new_pool_layer(layer_conv1,'pool1')
layer_relu1  =new_relu_layer(layer_pool1,'relu1')

layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=3, num_filters=32, name ="conv2")
layer_pool2 = new_pool_layer(layer_conv2,'pool2')
layer_relu2  =new_relu_layer(layer_pool2,'relu2')

layer_conv3, weights_conv3 = new_conv_layer(input=layer_relu2, num_input_channels=32, filter_size=3, num_filters=64, name ="conv3")
layer_pool3 = new_pool_layer(layer_conv3,'pool3')
layer_relu3  =new_relu_layer(layer_pool3,'relu3')

num_features = layer_relu3.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu3, [-1, num_features])
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128,name='fc1')

### Batch Normalisation
layer_bn = tf.contrib.layers.batch_norm(layer_fc1 ,center=True, scale=True)
layer_relu4 = new_relu_layer(layer_bn, name="relu3")
layer_fc2 = new_fc_layer(input=layer_relu4, num_inputs=128, num_outputs=10, name="fc2")


In [6]:
with tf.variable_scope('softmax'):
  y_pred = tf.nn.softmax(layer_fc2)
  y_pred_cls = tf.argmax(y_pred, axis=1)
    
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)
    
with tf.name_scope('opt'):
  optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
def printResult(epoch, numberOfEpoch, trainLoss, validationLoss, validationAccuracy):
    print("Epoch: {}/{}".format(epoch+1, numberOfEpoch),
         '\tTraining Loss: {:.3f}'.format(trainLoss),
         '\tValidation Loss: {:.3f}'.format(validationLoss),
         '\tAccuracy: {:.2f}%'.format(validationAccuracy*100))

num_epochs = 20
batchSize = 100

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

start = dt.now()
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())    
    for epoch in range(num_epochs):
        # training data & validation data
        train_x, val_x, train_y, val_y = train_test_split(X, Y,\
                                                      test_size = 0.2)   
        # training loss
        for i in range(0, len(train_x), 100):
            trainLoss, _= sess.run([cost, optimizer], feed_dict = {
                x: train_x[i: i+batchSize],
                y_true: train_y[i: i+batchSize]
            })
            
        # validation loss
        valAcc, valLoss = sess.run([accuracy, cost], feed_dict ={
            x: val_x,
            y_true: val_y,})
        
        
        # print out
        printResult(epoch, num_epochs, trainLoss, valLoss, valAcc)
        
print('running time:', dt.now() - start)

Epoch: 1/20 	Training Loss: 1.007 	Validation Loss: 0.994 	Accuracy: 77.21%
Epoch: 2/20 	Training Loss: 0.645 	Validation Loss: 0.616 	Accuracy: 85.21%
Epoch: 3/20 	Training Loss: 0.458 	Validation Loss: 0.455 	Accuracy: 88.73%
Epoch: 4/20 	Training Loss: 0.364 	Validation Loss: 0.351 	Accuracy: 92.61%
Epoch: 5/20 	Training Loss: 0.303 	Validation Loss: 0.270 	Accuracy: 94.67%
Epoch: 6/20 	Training Loss: 0.214 	Validation Loss: 0.198 	Accuracy: 97.58%
Epoch: 7/20 	Training Loss: 0.139 	Validation Loss: 0.157 	Accuracy: 97.82%
Epoch: 8/20 	Training Loss: 0.083 	Validation Loss: 0.121 	Accuracy: 98.67%
Epoch: 9/20 	Training Loss: 0.100 	Validation Loss: 0.086 	Accuracy: 99.15%
Epoch: 10/20 	Training Loss: 0.078 	Validation Loss: 0.066 	Accuracy: 99.52%
Epoch: 11/20 	Training Loss: 0.059 	Validation Loss: 0.045 	Accuracy: 99.88%
Epoch: 12/20 	Training Loss: 0.040 	Validation Loss: 0.037 	Accuracy: 99.76%
Epoch: 13/20 	Training Loss: 0.033 	Validation Loss: 0.032 	Accuracy: 100.00%
Epoch: 

In [8]:
## ======================== SqueezeNet =========================== ##

# define squeeze module
def squeeze(input, channels, layer_num):
    """
    Defines squeezed block for fire module.

    :param input: input tensor
    :param channels: number of output channels
    :param layer_num: layer number for naming purposes
    :return: output tensor convoluted with squeeze layer
    """
    layer_name = 'squeeze_' + str(layer_num)
    input_channels = input.get_shape().as_list()[3]

    with tf.name_scope(layer_name):
        weights = tf.Variable(tf.contrib.layers.xavier_initializer()([1, 1, input_channels, channels]))
        biases = tf.Variable(tf.zeros([1, 1, 1, channels]), name='biases')
        onebyone = tf.nn.conv2d(input, weights, strides=(1, 1, 1, 1), padding='VALID') + biases
        A = tf.nn.relu(onebyone)

    return A

# define expand module
def expand(input, channels_1by1, channels_3by3, layer_num):
    """
    Defines expand block for fire module.
    :param input: input tensor
    :param channels_1by1: number of output channels in 1x1 layers
    :param channels_3by3: number of output channels in 3x3 layers
    :param layer_num: layer number for naming purposes
    :return: output tensor convoluted with expand layer
    """

    layer_name = 'expand_' + str(layer_num)
    input_channels = input.get_shape().as_list()[3]

    with tf.name_scope(layer_name):
        weights1x1 = tf.Variable(tf.contrib.layers.xavier_initializer()([1, 1, input_channels, channels_1by1]))
        biases1x1 = tf.Variable(tf.zeros([1, 1, 1, channels_1by1]), name='biases')
        onebyone = tf.nn.conv2d(input, weights1x1, strides=(1, 1, 1, 1), padding='VALID') + biases1x1
        A_1x1 = tf.nn.relu(onebyone)

        weights3x3 = tf.Variable(tf.contrib.layers.xavier_initializer()([1, 1, input_channels, channels_3by3]))
        biases3x3 = tf.Variable(tf.zeros([1, 1, 1, channels_3by3]), name='biases')
        threebythree = tf.nn.conv2d(input, weights3x3, strides=(1, 1, 1, 1), padding='SAME') + biases3x3
        A_3x3 = tf.nn.relu(threebythree)

    return tf.concat([A_1x1, A_3x3], axis=3)


# define fire module
def fire_module(input, squeeze_channels, expand_channels_1by1, expand_channels_3by3, layer_num):
    """
    Train fire module. Fire module does not change input height and width, only depth.
    :param input: input tensor
    :param squeeze_channels: number of channels for 1x1 squeeze layer
    :param expand_channels_1by1: number of channels for 1x1 expand layer
    :param expand_channels_3by3: number of channels for 3x3 expand layer
    :param layer_num: number of layer for naming purposes only
    :return: a tensor of shape [input_height x input_width x expand_channels_1by1 * expand_channels_3by3]
    """
    with tf.name_scope('fire_' + str(layer_num)):
        squeeze_output = squeeze(input, squeeze_channels, layer_num)
        return expand(squeeze_output, expand_channels_1by1, expand_channels_3by3, layer_num)

In [9]:
input_height = input_width = 64
input_channels = 1
output_classes = 10
pooling_size=(1, 3, 3, 1)

input_image = tf.placeholder(tf.float32,shape=[None, input_height, input_width, 1],name='input_image')
labels = tf.placeholder(tf.float32, shape=[None,10])
labels_cls = tf.argmax(labels, axis=1)
in_training = tf.placeholder(tf.bool, shape=())

# layer 1 - conv 1
W_conv1 = tf.Variable(tf.contrib.layers.xavier_initializer()([7, 7, 1, 96]))
b_conv1 = tf.Variable(tf.zeros([1, 1, 1, 96]))
X_1 = tf.nn.conv2d(input_image, W_conv1, strides=(1, 2, 2, 1), padding='VALID') + b_conv1
A_1 = tf.nn.relu(X_1)

# layer 2 - maxpool
maxpool_1 = tf.nn.max_pool(A_1, ksize=pooling_size, strides=(1, 2, 2, 1), padding='VALID', name='maxpool_1')

# layer 3-5 - fire modules
fire_2 = fire_module(maxpool_1, 16, 64, 64, layer_num=2)
fire_3 = fire_module(fire_2, 16, 64, 64, layer_num=3)
fire_4 = fire_module(fire_3, 32, 128, 128, layer_num=4)

# layer 6 - maxpool
maxpool_4 = tf.nn.max_pool(fire_4, ksize=pooling_size, strides=(1, 2, 2, 1), padding='VALID', name='maxpool_4')

# layer 7-10 - fire modules
fire_5 = fire_module(maxpool_4, 32, 128, 128, layer_num=5)
fire_6 = fire_module(fire_5, 48, 192, 192, layer_num=6)
fire_7 = fire_module(fire_6, 48, 192, 192, layer_num=7)
fire_8 = fire_module(fire_7, 64, 256, 256, layer_num=8)

# layer 11 - maxpool
maxpool_8 = tf.nn.max_pool(fire_8, ksize=pooling_size, strides=(1, 2, 2, 1), padding='VALID', name='maxpool_8')

# layer 12 - fire 9 + dropout
fire_9 = fire_module(maxpool_8, 64, 256, 256, layer_num=9)
dropout_9 = tf.cond(in_training,lambda: tf.nn.dropout(fire_9, keep_prob=0.5),lambda: fire_9)

# layer 13 - final
W_conv10 = tf.Variable(tf.contrib.layers.xavier_initializer()([1, 1, 512, output_classes]))
b_conv10 = tf.Variable(tf.zeros([1, 1, 1, output_classes]))
conv_10 = tf.nn.conv2d(dropout_9, W_conv10, strides=(1, 1, 1,1), padding='VALID') + b_conv10            
A_conv_10 = tf.nn.relu(conv_10)

# avg pooling to get [1 x 1 x num_classes] must average over entire window oh H x W from input layer
_, H_last, W_last, _ = A_conv_10.get_shape().as_list()
pooled = tf.nn.avg_pool(A_conv_10, ksize=(1, H_last, W_last, 1), strides=(1, 1, 1, 1), padding='VALID')
logits = tf.squeeze(pooled, axis=[1, 2])

In [10]:
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)


# loss + optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(cost)


labels_cls = tf.cast(labels_cls,tf.float32)
y_pred_cls = tf.cast(y_pred_cls,tf.float32)
# accuracy
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred_cls, labels_cls), dtype=tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

num_epochs = 300
batchSize = 500
import time

def printResult(epoch, numberOfEpoch, trainLoss, validationLoss, validationAccuracy):
    print("Epoch: {}/{}".format(epoch, numberOfEpoch),
         '\tTraining Loss: {:.3f}'.format(trainLoss),
         '\tValidation Loss: {:.3f}'.format(validationLoss),
         '\tAccuracy: {:.2f}%'.format(validationAccuracy*100))

start = dt.now()
    
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs+1):
        # training data & validation data
        train_x, val_x, train_y, val_y = train_test_split(X, Y ,test_size = 0.2)  
        # training loss
        for i in range(0, len(train_x), 100):
            trainLoss, _= sess.run([cost, optimizer], feed_dict = {
                input_image: np.expand_dims(train_x[i: i+batchSize], axis=4),
                labels: train_y[i: i+batchSize],
                in_training: True
            })
            
        # validation loss
        valAcc, valLoss = sess.run([accuracy, cost], feed_dict ={
            input_image: np.expand_dims(val_x, axis=4),
            labels: val_y,
            in_training: True})
        
        # print out
        if(epoch%10 == 0):
            printResult(epoch, num_epochs, trainLoss, valLoss, valAcc)
            
print('running time:', dt.now() - start)

c:\users\ryan\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
c:\users\ryan\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


Epoch: 0/300 	Training Loss: 2.303 	Validation Loss: 2.303 	Accuracy: 10.18%
Epoch: 10/300 	Training Loss: 1.853 	Validation Loss: 1.859 	Accuracy: 23.76%
Epoch: 20/300 	Training Loss: 1.589 	Validation Loss: 1.581 	Accuracy: 38.91%
Epoch: 30/300 	Training Loss: 1.487 	Validation Loss: 1.252 	Accuracy: 50.79%
Epoch: 40/300 	Training Loss: 1.075 	Validation Loss: 1.158 	Accuracy: 53.45%
Epoch: 50/300 	Training Loss: 0.916 	Validation Loss: 1.001 	Accuracy: 59.39%
Epoch: 60/300 	Training Loss: 0.928 	Validation Loss: 1.067 	Accuracy: 57.94%
Epoch: 70/300 	Training Loss: 0.948 	Validation Loss: 0.864 	Accuracy: 65.45%
Epoch: 80/300 	Training Loss: 0.692 	Validation Loss: 0.801 	Accuracy: 69.21%
Epoch: 90/300 	Training Loss: 0.720 	Validation Loss: 0.676 	Accuracy: 72.61%
Epoch: 100/300 	Training Loss: 0.692 	Validation Loss: 0.826 	Accuracy: 68.36%
Epoch: 110/300 	Training Loss: 0.478 	Validation Loss: 0.649 	Accuracy: 75.52%
Epoch: 120/300 	Training Loss: 0.511 	Validation Loss: 0.592 	A